<a href="https://colab.research.google.com/github/Pras89tyo/Face_replacement/blob/main/FaceReplacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Package

In [1]:
# Install required packages
!pip install insightface==0.7.3
!pip install onnxruntime
!pip install opencv-python
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 68.0 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1064898 sha256=6b41741883f0a2925766f8c27332419725abed94f82f2263f44625747a9a4949
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00


import package

In [2]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import requests
import os

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Download model

In [3]:
def download_model():
    # Download the model
    url = "https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx"
    response = requests.get(url)

    # Save the model
    model_path = "inswapper_128.onnx"
    with open(model_path, "wb") as f:
        f.write(response.content)
    return model_path

In [4]:
def face_swap(source_img_path, target_img_path, output_path):
    # Initialize face analyzer
    app = FaceAnalysis(name='buffalo_l')
    app.prepare(ctx_id=0, det_size=(640, 640))

    # Load the swapper
    swapper = insightface.model_zoo.get_model('inswapper_128.onnx')

    # Read images
    source_img = cv2.imread(source_img_path)
    target_img = cv2.imread(target_img_path)

    # Get face information
    source_faces = app.get(source_img)
    target_faces = app.get(target_img)

    # Check if faces are detected
    if len(source_faces) == 0 or len(target_faces) == 0:
        raise ValueError("No faces detected in one or both images")

    # Use the first detected face from each image
    source_face = source_faces[0]

    # Swap faces for all detected faces in target image
    result_img = target_img.copy()
    for target_face in target_faces:
        result_img = swapper.get(result_img, target_face, source_face, paste_back=True)

    # Save result
    cv2.imwrite(output_path, result_img)
    return output_path


In [5]:
# Example usage
def main():
    # Download model
    model_path = download_model()

    # Define paths
    source_path = "/content/face.jpg"  # Path to source face image
    target_path = "/content/antony-3.jpeg"  # Path to target image
    output_path = "result.jpg"        # Path for output image

    # Perform face swap
    try:
        result = face_swap(source_path, target_path, output_path)
        print(f"Face swap completed successfully. Result saved to: {output_path}")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    main()

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:04<00:00, 58338.26KB/s]
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
Applied prov

/usr/local/lib/python3.11/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


Face swap completed successfully. Result saved to: result.jpg
